In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fast_gen import *
from learner import *
from pt_models import *
from dataset_pt import *
from sgdr_pt import *

from planet import *

In [ ]:
project_name = 

In [ ]:
path=

In [ ]:
arch=resnet50
size=64
bs=64
metrics=[f2]

In [ ]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

In [ ]:
no_of_rows = len(list(open(f'{path}train_v2.csv')))-1

In [ ]:
val_idxs = get_cv_idxs(no_of_rows)

In [ ]:
data = get_data(size)

In [ ]:
x, y = next(iter(data.val_dl))

In [ ]:
y

In [ ]:
list(zip(data.classes, y[0]))

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] )

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] * 1.3 ) # TIP : Multiply by a value if the image is too hazy

In [ ]:
data = get_data(size)

In [ ]:
data = data.resize(int(size*1.3))

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] )

## Train model

In [ ]:
learn = ConvLearner.pretrained(arch, data, bs=bs, metrics=metrics)

In [ ]:
lrf = learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learning_rate = 

##### As sat images are not part of imagnet, we divide by smaller numbers as we want to train the entire network from scratch. Well kind of, we initially use the weights from imagnet model as something is better than nothing and also, imagnet initial layers actually detect basic structures like edges etc.

In [ ]:
learning_rates = np.array(learning_rate/9, learning_rate/3, learning_rate) 

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit(learning_rate, 3, cycle_len=1, cycle_mult=2, metrics=metrics)

In [ ]:
now = Datet
learn.save(f'{project_name}_last_layer_{sz}')

In [ ]:
learn.sched.plot_loss()

### Increase pic size

In [ ]:
size = 128

In [ ]:
learn.set_data(get_data(size))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, metrics=metrics)
learn.sched.plot_loss()

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2, metrics=metrics)
learn.sched.plot_loss()
learn.save(f'{project_name}_last_layer_{sz}')

In [ ]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

In [ ]:
f2(preds,y)